# Reduced Gravity

## Classes and modules

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

GPU Ocean-modules:

In [ ]:
from gpuocean.SWEsimulators import CDKLM16
from gpuocean.utils import Common, IPythonMagic, NetCDFInitialization

In [ ]:
%cuda_context_handler gpu_ctx

Plot utils

In [ ]:
from IPython.display import clear_output
from matplotlib import animation, rc
plt.rcParams["animation.html"] = "jshtml"
from mpl_toolkits.axes_grid1 import make_axes_locatable

from gpuocean.utils import PlotHelper
from gpuocean.utils.NetCDFInitialization import depth_integration

def plotSolution(fig, 
                 eta, hu, hv, h, dx, dy, 
                 t, comment,
                 h_min=-0.5, h_max=0.5, 
                 uv_min=-0.3, uv_max=0.3,
                 calc_uv = False, 
                 add_extra=False,
                 reduced_gravity_interface=None,
                 ax=None, sp=None):


    from datetime import timedelta
    fig.suptitle("Time = {:0>8} ({:s})".format(str(timedelta(seconds=int(t))), comment), 
                 fontsize=18,
                 horizontalalignment='left')
    
    ny, nx = eta.shape
    domain_extent = [0, nx*dx, 0, ny*dy]
    
    x_plots = 3
    y_plots = 1
    if (add_extra == True):
        x_plots=3
        y_plots=2
    
    V_max = 3 * (uv_max-uv_min) / np.max(h)
    R_min = -V_max/2000
    R_max = V_max/2000
   
    huv_label = ["hu","hv"]
    if calc_uv:
        """plotting actually u and v"""
        if reduced_gravity_interface is None or reduced_gravity_interface == 0.0:
            hu = hu/(h+eta)
            hv = hv/(h+eta)
        else:
            hu = hu/(reduced_gravity_interface+eta)
            hv = hv/(reduced_gravity_interface+eta)
        uv_min = -0.3
        uv_max = +0.3
        huv_label = ["u","v"]

    if (ax is None):
        ax = [None]*x_plots*y_plots
        sp = [None]*x_plots*y_plots

        uv_cmap = plt.cm.coolwarm
        uv_cmap.set_bad("grey", alpha = 1.0)
        
        h_cmap = plt.cm.coolwarm
        h_cmap.set_bad("grey", alpha = 1.0)

        ax[0] = plt.subplot(y_plots, x_plots, 1)
        sp[0] = ax[0].imshow(eta, interpolation="none", origin='lower', 
                             cmap=h_cmap, 
                             vmin=h_min, vmax=h_max, 
                             extent=domain_extent)
        plt.axis('image')
        plt.title("$\zeta{}$")
        divider0 = make_axes_locatable(ax[0])
        cax0 = divider0.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[0],cax=cax0)


        ax[1] = plt.subplot(y_plots, x_plots, 2)
        sp[1] = ax[1].imshow(hu, interpolation="none", origin='lower', 
                            cmap=uv_cmap, 
                            vmin=uv_min, vmax=uv_max, 
                            extent=domain_extent)
        plt.axis('image')
        plt.title("$"+huv_label[0]+"$")
        divider1 = make_axes_locatable(ax[1])
        cax1 = divider1.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[1],cax=cax1)



        ax[2] = plt.subplot(y_plots, x_plots, 3)
        sp[2] = ax[2].imshow(hv, interpolation="none", origin='lower', 
                             cmap=uv_cmap, 
                             vmin=uv_min, vmax=uv_max, 
                             extent=domain_extent)
        plt.axis('image')
        plt.title("$"+huv_label[1]+"$")
        divider2 = make_axes_locatable(ax[2])
        cax2 = divider2.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(sp[2],cax=cax2)

        plt.tight_layout()
            
    else:        
        #Update plots
        fig.sca(ax[0])
        sp[0].set_data(eta)
        
        fig.sca(ax[1])
        sp[1].set_data(hu)
        
        fig.sca(ax[2])
        sp[2].set_data(hv)
        
        if (add_extra == True):
            V = PlotHelper.genVelocity(h, hu, hv)
            fig.sca(ax[3])
            sp[3].set_data(V)

            R = PlotHelper.genColors(h, hu/dx, hv/dy, plt.cm.seismic, R_min, R_max)
            fig.sca(ax[4])
            sp[4].set_data(R)
    
    return ax, sp

def ncAnimation(filename, movie_frames=None, create_movie=True, fig=None, x0=0, x1=-1, y0=0, y1=-1, reduced_gravity_interface=None, **kwargs):
    #Create figure and plot initial conditions
    if fig is None:
        fig = plt.figure(figsize=(24, 4))

    try:
        ncfile = Dataset(filename)
        try:
            x = ncfile.variables['x'][:]
        except:
            x = ncfile.variables['X'][x0:x1]
        try:
            y = ncfile.variables['y'][:]
        except:
            y = ncfile.variables['Y'][y0:y1]
        t = ncfile.variables['time'][:]

        try:
            H_m = ncfile.variables['Hm'][:,:]
        except:
            H_m = ncfile.variables['h'][y0:y1,x0:x1]
        try:
            eta = ncfile.variables['eta'][:,:,:]
        except:
            eta = ncfile.variables['zeta'][:,y0:y1,x0:x1]
        try:
            hu = ncfile.variables['hu'][:,:,:]
            calc_uv = True
        except:
            if reduced_gravity_interface is None:
                hu = ncfile.variables['ubar'][:,y0:y1,x0:x1]
                calc_uv = False
            else:
                if reduced_gravity_interface > 0.0:
                    hu = np.ma.zeros(eta.shape)
                    for t_idx in range(len(t)):
                        hu[t_idx] = depth_integration(ncfile, reduced_gravity_interface, x0, x1, y0, y1, "u", timestep_index=t_idx)
                    calc_uv = True
                else:
                    hu = ncfile.variables['u'][:,0,y0:y1,x0:x1]
                    calc_uv = False
        try:
            hv = ncfile.variables['hv'][:,:,:]
        except:
            if reduced_gravity_interface is None:
                hv = ncfile.variables['vbar'][:,y0:y1,x0:x1]
            else:
                if reduced_gravity_interface > 0.0:
                    hv = np.ma.zeros(eta.shape)
                    for t_idx in range(len(t)):
                        hv[t_idx] = depth_integration(ncfile, reduced_gravity_interface, x0, x1, y0, y1, "v", timestep_index=t_idx)
                else:
                    hv = ncfile.variables['v'][:,0,y0:y1,x0:x1]
                    
    except Exception as e:
        raise e
    finally:
        ncfile.close()


    if movie_frames is None:
        movie_frames = len(t)

    dx = x[1] - x[0]
    dy = y[1] - y[0]
    
    progress = Common.ProgressPrinter(5)

    if (create_movie):
        ax, sp = plotSolution(fig, 
                              eta[0],
                              hu[0],
                              hv[0],
                              H_m+eta[0],
                              dx, dy, 
                              t[0], "Initalised from Mixed Layer Daily-Average",
                              calc_uv=calc_uv,
                              reduced_gravity_interface=reduced_gravity_interface,
                              **kwargs)
    else:
        ax, sp = plotSolution(fig, 
                              eta[-1],
                              hu[-1],
                              hv[-1],
                              H_m+eta[-1],
                              dx, dy, 
                              t[-1], filename,
                              calc_uv=calc_uv,
                              **kwargs)
        return

    #Helper function which simulates and plots the solution    
    def animate(i):
        t_now = t[0] + (i / (movie_frames-1)) * (t[-1] - t[0]) 

        k = np.searchsorted(t, t_now)
        if (k >= eta.shape[0]):
            k = eta.shape[0] - 1
        j = max(0, k-1)
        if (j == k):
            k += 1
        s = (t_now - t[j]) / (t[k] - t[j])

        plotSolution(fig, 
                        (1-s)*eta[j] + s*eta[k], 
                        (1-s)*hu[j]  + s*hu[k], 
                        (1-s)*hv[j]  + s*hv[k], 
                        H_m+(1-s)*eta[j] + s*eta[k], 
                        dx, dy, 
                        t_now, "Initalised from Mixed Layer Daily-Average",
                        calc_uv=calc_uv,
                        reduced_gravity_interface=reduced_gravity_interface,
                        **kwargs, ax=ax, sp=sp)

        clear_output(wait = True)
        #print(progress.getPrintString(i / (movie_frames-1)))

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(movie_frames), interval=250)
    plt.close(fig)
    
    return anim


## Selecting Area

As initial and boundary conditions to the simulation, we use data from the Norkyst800 s-level files:

In [ ]:
casename = 'skagerrak'

from importlib import reload
from gpuocean.utils import NetCDFInitialization
reload(NetCDFInitialization)
case_info = NetCDFInitialization.getCaseLocation(casename)
case_info.pop("name")

In [ ]:
locals().update(case_info)

## Construct initial conditions

In [ ]:
#s_source_url = "C:\\Users\\florianb\\Downloads\\ocean_his.an.20190716.nc"
s_source_url = "/sintef/data/NorKyst800/ocean_avg.an.20190716.nc"

In [ ]:
from importlib import reload 
reload(NetCDFInitialization)
data_args = NetCDFInitialization.getInitialConditions(s_source_url, x0, x1, y0, y1, download_data=False, norkyst_data=False)
NetCDFInitialization.removeMetadata(data_args).keys()

In [ ]:
data_args.pop("wind_stress")
# from gpuocean.utils import WindStress
# data_args["wind_stress"] = WindStress.WindStress(X=[np.array([[100]])],Y=[np.array([[0]])])

In [ ]:
data_args["f"] = 1.2e-4
data_args["latitude"] = None

In [ ]:
temp_data_args = NetCDFInitialization.getInitialConditions(s_source_url, x0-1, x1+1, y0-1, y1+1, download_data=False, norkyst_data=False)

temp_sim_args = {
    "gpu_ctx": gpu_ctx,
    "write_netcdf":True,
    "dt":0.0
     }
     
temp_sim = CDKLM16.CDKLM16(**temp_sim_args, **NetCDFInitialization.removeMetadata(temp_data_args))

In [ ]:
#Calculate derivatives 
DetaDy, DetaDx = np.gradient(temp_data_args["eta0"])
DetaDx = DetaDx / data_args["dx"]
DetaDy = DetaDy / data_args["dy"]

#Get north and east vectors
angle = temp_data_args["angle"] #one halo needed
north = [np.sin(angle), np.cos(angle)]
east = [np.cos(angle), -np.sin(angle)]

#Calculat derivatives towards north and east for eta
DetaDeast = east[0]*DetaDx + east[1]*DetaDy
DetaDnorth = north[0]*DetaDx + north[1]*DetaDy

#Calculate h
h = temp_data_args["eta0"] + temp_sim.downloadBathymetry()[1]

#Get northward and eastward momentums
hu_east = -data_args["g"]/data_args["f"] * h * DetaDnorth
hv_north = data_args["g"]/data_args["f"] * h * DetaDeast

# Analytic inversion of 2x2 linear system
det = east[0]*north[1] - east[1]*north[0]

hu0 = 1/det * ( north[1] * hu_east - east[1] * hv_north )
hv0 = 1/det * ( -north[0] * hu_east + east[0] * hv_north)

In [ ]:
hu0.data[hu0.mask == True] = 0.0
hv0.data[hv0.mask == True] = 0.0

In [ ]:
data_args["hu0"] = np.ma.array(hu0.data[1:-1,1:-1], mask = hu0.mask[1:-1,1:-1])
data_args["hv0"] = np.ma.array(hv0.data[1:-1,1:-1], mask = hv0.mask[1:-1,1:-1])

In [ ]:
data_args["boundary_conditions_data"].north.hu = np.array([hu0[-1,1:-1]], dtype=np.float32)
data_args["boundary_conditions_data"].south.hu = np.array([hu0[0,1:-1]], dtype=np.float32)
data_args["boundary_conditions_data"].east.hu = np.array([hu0[1:-1,-1]], dtype=np.float32)
data_args["boundary_conditions_data"].west.hu = np.array([hu0[1:-1,0]], dtype=np.float32)

data_args["boundary_conditions_data"].north.hv = np.array([hv0[-1,1:-1]], dtype=np.float32)
data_args["boundary_conditions_data"].south.hv = np.array([hv0[0,1:-1]], dtype=np.float32)
data_args["boundary_conditions_data"].east.hv = np.array([hv0[1:-1,-1]], dtype=np.float32)
data_args["boundary_conditions_data"].west.hv = np.array([hv0[1:-1,0]], dtype=np.float32)

## Simulation

In [ ]:
sim_args = {
    "gpu_ctx": gpu_ctx,
    "dt": 0.0,
    "write_netcdf":True,
    "rk_order":1
     }
    
from importlib import reload
reload(CDKLM16)

sim = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args))
#sim = CDKLM16.CDKLM16(**sim_args, **NetCDFInitialization.removeMetadata(data_args), static_eta=True)

Check the geostrophic balance!!!

In [ ]:
from gpuocean.utils import OceanographicUtilities
geos, geos_x, geos_y = OceanographicUtilities.calcGeostrophicBalance(data_args["eta0"], sim.downloadBathymetry()[1], data_args["hu0"], data_args["hv0"], data_args["angle"], data_args["f"], data_args["dx"], data_args["dy"], g=data_args["g"], norkyst=True)

plt.figure(figsize=(32,4))
plt.suptitle("Geostrophic balance (hu)")
plt.subplot(1,3,1)
plt.title("Coriolis contribution")
plt.imshow(geos_x[0], origin='lower', vmax=0.02, vmin=-0.02)
plt.colorbar(shrink=0.7)

plt.subplot(1,3,2)
plt.title("Momentum contribution")
plt.imshow(geos_x[1], origin='lower', vmax=0.02, vmin=-0.02)
plt.colorbar(shrink=0.7)

plt.subplot(1,3,3)
plt.title("Geostrophic imbalance")
plt.imshow(geos[0], origin='lower', vmax=2e-9, vmin=-2e-9)
plt.colorbar(shrink=0.7)

plt.figure(figsize=(32,4))
plt.suptitle("Geostrophic balance (hv)")
plt.subplot(1,3,1)
plt.title("Coriolis contribution")
plt.imshow(geos_y[0], origin='lower', vmax=0.02, vmin=-0.02)
plt.colorbar(shrink=0.7)

plt.subplot(1,3,2)
plt.title("Momentum contribution")
plt.imshow(geos_y[1], origin='lower', vmax=0.02, vmin=-0.02)
plt.colorbar(shrink=0.7)

plt.subplot(1,3,3)
plt.title("Geostrophic imbalance")
plt.imshow(geos[1], origin='lower', vmax=2e-9, vmin=-2e-9)
plt.colorbar(shrink=0.7)


Animation

In [ ]:
frame = ncAnimation(sim.sim_writer.output_file_name, create_movie=False)
frame

In [ ]:
for hour in range(6*24):
    sim.step(600, apply_stochastic_term=False)

In [ ]:
anim = ncAnimation(sim.sim_writer.output_file_name)
anim

Very clear gravitational waves in the first hours, then stable